## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Eureka,US,40.8021,-124.1637,54.00,72,100,3.00,overcast clouds
1,1,Skibbereen,IE,51.5500,-9.2667,33.80,64,75,20.71,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,64.40,59,40,4.61,scattered clouds
3,3,Bar-Le-Duc,FR,48.7500,5.1667,32.00,80,90,6.91,overcast clouds
4,4,Tucuma,BR,-0.1833,-50.0500,78.82,87,88,7.83,moderate rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 65


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                     (city_data_df["Max Temp"] <= max_temp)]

In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df
clean_df = clean_df.rename(columns={'Description' : 'Current Description'})
clean_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Eureka,US,40.8021,-124.1637,54.00,72,100,3.00,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,64.40,59,40,4.61,scattered clouds
5,5,Amnat Charoen,TH,15.9167,104.7500,63.72,83,100,17.49,overcast clouds
11,11,Batemans Bay,AU,-35.7167,150.1833,60.01,94,100,1.99,overcast clouds
12,12,Hobart,AU,-42.8794,147.3294,50.00,81,20,5.75,few clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Eureka,US,54.00,overcast clouds,40.8021,-124.1637,
2,Ushuaia,AR,64.40,scattered clouds,-54.8000,-68.3000,
5,Amnat Charoen,TH,63.72,overcast clouds,15.9167,104.7500,
11,Batemans Bay,AU,60.01,overcast clouds,-35.7167,150.1833,
12,Hobart,AU,50.00,few clouds,-42.8794,147.3294,
16,Barcelona,ES,57.00,few clouds,41.3888,2.1590,
30,Bluff,NZ,52.41,overcast clouds,-46.6000,168.3333,
32,Ribeira Grande,PT,62.60,broken clouds,38.5167,-28.7000,
34,Busselton,AU,54.00,overcast clouds,-33.6500,115.3333,
40,Gat,IL,64.00,clear sky,31.6100,34.7642,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print('Hotel not found...')

Hotel not found...
Hotel not found...
Hotel not found...
Hotel not found...


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Eureka,US,54.00,overcast clouds,40.8021,-124.1637,Eureka Inn Trademark By Wyndham
2,Ushuaia,AR,64.40,scattered clouds,-54.8000,-68.3000,Albatros Hotel
5,Amnat Charoen,TH,63.72,overcast clouds,15.9167,104.7500,หอพักนักศึกษา
11,Batemans Bay,AU,60.01,overcast clouds,-35.7167,150.1833,Batemans Bay Marina Resort
12,Hobart,AU,50.00,few clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))